In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import twokenize
from nltk.tokenize import TweetTokenizer
from FeatureFunctions import getfeatures
from FeatureFunctions import helper
from sklearn.externals import joblib
from gensim.models import Word2Vec
from importlib import reload

import sklearn
import csv


### Import the classifier
import sys
sys.path.insert(0, 'libsvm')

from svmutil import *

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [2]:
# whichtask = input("Which task to you want to do (A/B): ")
whichtask = "A"

In [3]:
### Reading train and test files

datafile = "datasets/train/SemEval2018-T3-train-task"+whichtask+"_emoji.txt"
# datafile = "datasets/train/SemEval2018-T3-train-task"+whichtask+"_emoji_ironyHashtags.txt"

trainingdata = pd.read_csv(datafile, delimiter = "\t",  quoting=csv.QUOTE_NONE, header=0)
trainingdata = trainingdata[['Label','Tweet text']]

testfile = 'datasets/goldtest_Task'+whichtask+'/SemEval2018-T3_gold_test_task'+whichtask+'_emoji.txt'
testdata = pd.read_csv(testfile, sep="\t",  quoting=csv.QUOTE_NONE, header=0)
testdata = testdata[['Label','Tweet text']]

In [4]:
reload(helper)

<module 'FeatureFunctions.helper' from '/root/Documents/TU_Delft/InformationRetrieval/NLP/InformationRetrievalNLP/FeatureFunctions/helper.py'>

In [5]:
train_posfeatures = helper.getPOSfeatures("FeatureFunctions/POStrain.txt")
test_posfeatures = helper.getPOSfeatures("FeatureFunctions/POStest.txt")

In [6]:
### Get lexical features
# training_features
lexical_training_features, unicount_vect, bicount_vect, tricount_vect, fourcount_vect = getfeatures.getlexical(trainingdata, 'Tweet text')
x_small = lexical_training_features[['PunctuationFlood', 'CharFlood', 'CapitalizedCount', 'HashtagCount', 'Hashtag2WordRatio', 'TweetCharLength', 'TweetWordLength', 'EmojiCount', 'FinalPunctuation']].values
x_lexical = np.array(lexical_training_features.apply(lambda row: sum([row['CharFourgramVector'], row['CharTrigramVector'],row['BigramVector']], []), axis=1).values.tolist())
x_lexical = np.hstack((x_small, x_lexical))

# train_bow = lexical_training_features['UnigramVector'].values.tolist()

# test_features
lexical_test_features, unicount_vect, bicount_vect, tricount_vect, fourcount_vect = getfeatures.getlexical(testdata, 'Tweet text', unicount_vect, bicount_vect, tricount_vect, fourcount_vect)
test_x_small = lexical_test_features[['PunctuationFlood', 'CharFlood', 'CapitalizedCount', 'HashtagCount', 'Hashtag2WordRatio', 'TweetCharLength', 'TweetWordLength', 'EmojiCount', 'FinalPunctuation']].values
test_lexical_x = np.array(lexical_test_features.apply(lambda row: sum([row['CharFourgramVector'], row['CharTrigramVector'],row['BigramVector']], []), axis=1).values.tolist())
test_lexical_x = np.hstack((test_x_small, test_lexical_x))

train_bow = np.array(lexical_training_features['UnigramVector'].values.tolist())
test_bow = np.array(lexical_test_features['UnigramVector'].values.tolist())

lexical_training_features = []
lexical_test_features = []




In [7]:
### Load the classifier
kmeansmodel = joblib.load("FeatureFunctions/kmeansclass.model")
w2vmodel = Word2Vec.load("FeatureFunctions/word2vec.model")

train_bags = helper.allbags(trainingdata['Tweet text'])
word2vecfeatures_train = helper.onehotwordclusters(train_bags, kmeansmodel, w2vmodel).astype(int)

test_bags = helper.allbags(testdata['Tweet text'])
word2vecfeatures_test = helper.onehotwordclusters(test_bags, kmeansmodel, w2vmodel).astype(int)

In [8]:
reload(getfeatures)

### Get sentiment features
train_sentiment_x = getfeatures.getaffinfeats(trainingdata['Tweet text'])
test_sentiment_x = getfeatures.getaffinfeats(testdata['Tweet text'])


In [14]:
### Combining features
iterations = int(input("How many iterations: "))
# nuSVC = int(input("Use nusvc? (1):"))
nuSVC = 0
heldoutset = int(input("Do you want to use a heldout set (1=heldout, 0=crossvalidate): "))
whichtype = int(input("Which type to test: "))
results = []

for i in range(iterations):
    print()
    print("-------------------------------------------------------------------")
    print("Iteration " + str(i+1) + "/" + str(iterations))
    print("-------------------------------------------------------------------")
    print()
    
    ### Selecting the actual training samples (Do some crossover here )
    if heldoutset:
        trainingdata_result = trainingdata
        testdata_result = testdata
        resulting_train_index = range(len(trainingdata))
        resulting_test_index = range(len(testdata_result))
    else:
        ## Crossvalidating
        resulting_train_index, resulting_test_index = helper.getindices(trainingdata, 4, 1) # Get 80-20
        trainingdata_result = trainingdata.loc[resulting_train_index]
        testdata_result = trainingdata.loc[resulting_test_index]
    
    final_train_x = []
    final_test_x = []

    if heldoutset:
        ## BoW
        if whichtype == 1:
            final_train_x = train_bow[resulting_train_index]
            final_test_x = test_bow[resulting_test_index]
        ## Lexical
        if whichtype == 2:
            final_train_x = np.hstack((train_bow[resulting_train_index], x_lexical[resulting_train_index]))
            final_test_x = np.hstack((test_bow[resulting_test_index], test_lexical_x[resulting_test_index]))
        ## Syntactic
        if whichtype == 3:
            final_train_x = train_posfeatures[resulting_train_index]
            final_test_x = test_posfeatures[resulting_test_index]
        ## Sentiment
        if whichtype == 4:
            final_train_x = train_sentiment_x[resulting_train_index]
            final_test_x = test_sentiment_x[resulting_test_index]
        ## Word2Vec
        if whichtype == 5:
            final_train_x = word2vecfeatures_train[resulting_train_index]
            final_test_x = word2vecfeatures_test[resulting_test_index]
        ## Combined
        if whichtype == 6:
            final_train_x = np.hstack((train_posfeatures[resulting_train_index], word2vecfeatures_train[resulting_train_index],train_bow[resulting_train_index], train_sentiment_x[resulting_train_index], x_lexical[resulting_train_index]))
            final_test_x = np.hstack((test_posfeatures[resulting_test_index], word2vecfeatures_test[resulting_test_index], test_bow[resulting_test_index], test_sentiment_x[resulting_test_index], test_lexical_x[resulting_test_index]))
    else:
        ## Crossvalidating
        ## BoW
        if whichtype == 1:
            final_train_x = train_bow[resulting_train_index]
            final_test_x = train_bow[resulting_test_index]
        ## Lexical
        if whichtype == 2:
            final_train_x = np.hstack((train_bow[resulting_train_index], x_lexical[resulting_train_index]))
            final_test_x = np.hstack((train_bow[resulting_test_index], x_lexical[resulting_test_index]))
        ## Syntactic
        if whichtype == 3:
            final_train_x = train_posfeatures[resulting_train_index]
            final_test_x = train_posfeatures[resulting_test_index]
        ## Sentiment
        if whichtype == 4:
            final_train_x = train_sentiment_x[resulting_train_index]
            final_test_x = train_sentiment_x[resulting_test_index]
        ## Word2Vec
        if whichtype == 5:
            final_train_x = word2vecfeatures_train[resulting_train_index]
            final_test_x = word2vecfeatures_train[resulting_test_index]
        ## Combined
        if whichtype == 6:
            final_train_x = np.hstack((train_posfeatures[resulting_train_index], word2vecfeatures_train[resulting_train_index], train_bow[resulting_train_index], train_sentiment_x[resulting_train_index], x_lexical[resulting_train_index]))
            final_test_x = np.hstack((train_posfeatures[resulting_test_index], word2vecfeatures_train[resulting_test_index], train_bow[resulting_test_index], train_sentiment_x[resulting_test_index], x_lexical[resulting_test_index]))


    print("###########")
    print("Training:")
    print(final_train_x.shape)
    print()
    ### Train and get train error
    y = trainingdata_result['Label'].tolist()
    prob  = svm_problem(y, final_train_x)
    if nuSVC:
        param = svm_parameter('-t 2 -s 1')
    else:
        param = svm_parameter('-t 2 -c 8 -g ' + str(2**-11))
    m = svm_train(prob, param)
    p_label, p_acc, p_val = svm_predict(y, final_train_x, m)
    ACC, MSE, SCC = evaluations(y, p_label)

    print()
    print("###########")
    print("Testing:")
    print(final_test_x.shape)
    print()


    ### Get the test
    test_y = testdata_result['Label'].tolist()

    test_p_label, test_p_acc, test_p_val = svm_predict(test_y, final_test_x, m)
    test_ACC, test_MSE, test_SCC = evaluations(test_y, test_p_label)
    print(sklearn.metrics.classification_report(test_y, test_p_label, digits=4))

    if whichtask == "A":
        p, r, f = helper.precision_recall_fscore(test_y, test_p_label, beta=1, labels=[0,1], pos_label=1)
    elif whichtask == "B":
        p, r, f = helper.precision_recall_fscore(test_y, test_p_label, beta=1, labels=[0,1,2,3])

    print("Precision: " + str(p))
    print("Recall: " + str(r))
    print("F1-score: " + str(f))
    
    results.append([test_ACC, p*100, r*100, f*100])

    y_actu = pd.Series(test_y, name='Actual')
    y_pred = pd.Series(test_p_label, name='Predicted')
    df_confusion = pd.crosstab(y_actu, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

    print()
    print()
    print(df_confusion)
    

### Write output
mean_results = np.mean(np.array(results), axis=0)
std_results = 2*np.std(np.array(results), axis=0)
min_results = np.min(np.array(results), axis=0)
max_results = np.max(np.array(results), axis=0)

if whichtype == 1:
    sys.stdout.write("BoW results")
elif whichtype == 2:
    sys.stdout.write("Lexical results")
elif whichtype == 3:
    sys.stdout.write("Syntactical results")
elif whichtype == 4:
    sys.stdout.write("Semantic results")
elif whichtype == 5:
    sys.stdout.write("Word2Vec results")
elif whichtype == 6:
    sys.stdout.write("Combined results")
    
### stdout
print(" with " + str(iterations) + " iterations: ")
for result_i in range(len(mean_results)):
    sys.stdout.write("{:.2f}".format(mean_results[result_i]))
    sys.stdout.write(" ±" + "{:.2f}".format(std_results[result_i]))
    sys.stdout.write(" -" + "{:.2f}".format(min_results[result_i]))
    sys.stdout.write(" +" + "{:.2f}".format(max_results[result_i]) + "\t")
    
    
### file out
with open("Results/resultswrite.txt", "a") as outfile:
    if whichtype == 1:
        outfile.write("BoW results")
    elif whichtype == 2:
        outfile.write("Lexical results")
    elif whichtype == 3:
        outfile.write("Syntactical results")
    elif whichtype == 4:
        outfile.write("Semantic results")
    elif whichtype == 5:
        outfile.write("Word2Vec results")
    elif whichtype == 6:
        outfile.write("Combined results")

    ### stdout
    outfile.write(" with " + str(iterations) + " iterations: \n")
    for result_i in range(len(mean_results)):
        outfile.write("{:.2f}".format(mean_results[result_i]))
        outfile.write(" ±" + "{:.2f}".format(std_results[result_i]))
        outfile.write(" -" + "{:.2f}".format(min_results[result_i]))
        outfile.write(" +" + "{:.2f}".format(max_results[result_i]) + "\t")
        
    outfile.write("\n\n")
print()

How many iterations: 10
Do you want to use a heldout set (1=heldout, 0=crossvalidate): 0
Which type to test: 6

-------------------------------------------------------------------
Iteration 1/10
-------------------------------------------------------------------

###########
Training:
(3058, 96217)

Accuracy = 96.501% (2951/3058) (classification)

###########
Testing:
(762, 96217)

Accuracy = 90.0262% (686/762) (classification)
              precision    recall  f1-score   support

           0     0.9472    0.8478    0.8947       381
           1     0.8622    0.9528    0.9052       381

   micro avg     0.9003    0.9003    0.9003       762
   macro avg     0.9047    0.9003    0.9000       762
weighted avg     0.9047    0.9003    0.9000       762

Precision: 0.8622327790973872
Recall: 0.952755905511811
F1-score: 0.9052369077306732


Predicted  0.0  1.0  All
Actual                  
0          323   58  381
1           18  363  381
All        341  421  762

----------------------------

In [8]:
if heldoutset:
    ### Evaluate results per category (Sectie 5.2)

    testfileB = 'datasets/goldtest_TaskB/SemEval2018-T3_gold_test_taskB_emoji.txt'
    testdataB = pd.read_csv(testfileB, sep="\t",  quoting=csv.QUOTE_NONE, header=0)
    testdataB = testdataB[['Label','Tweet text']]

    ## Getting category information
    used_testB = testdataB.loc[resulting_test_index]
    used_testB.reset_index(inplace=True, drop=True)

    ## Nonirony
    print("--- 0 Not ironic")
    predB_nonirony = sum((used_testB['Label'] == 0) & (y_pred == 0))
    testB_nonirony = sum((used_testB['Label'] == 0))
    print(str(predB_nonirony) + "/" + str(testB_nonirony))
    print("{:.2f}%".format(predB_nonirony/testB_nonirony * 100))
    print()

    ## 1
    print("--- 1 Ironic by clash")
    predB_nonirony = sum((used_testB['Label'] == 1) & (y_pred == 1))
    testB_nonirony = sum((used_testB['Label'] == 1))
    print(str(predB_nonirony) + "/" + str(testB_nonirony))
    print("{:.2f}%".format(predB_nonirony/testB_nonirony * 100))
    print()

    ## 2
    print("--- 2 Situational irony")
    predB_nonirony = sum((used_testB['Label'] == 2) & (y_pred == 1))
    testB_nonirony = sum((used_testB['Label'] == 2))
    print(str(predB_nonirony) + "/" + str(testB_nonirony))
    print("{:.2f}%".format(predB_nonirony/testB_nonirony * 100))
    print()

    ## 3
    print("--- 3 Other irony")
    predB_nonirony = sum((used_testB['Label'] == 3) & (y_pred == 1))
    testB_nonirony = sum((used_testB['Label'] == 3))
    print(str(predB_nonirony) + "/" + str(testB_nonirony))
    print("{:.2f}%".format(predB_nonirony/testB_nonirony * 100))
    print()


--- 0 Not ironic
473/473
100.00%

--- 1 Ironic by clash
0/164
0.00%

--- 2 Situational irony
0/85
0.00%

--- 3 Other irony
0/62
0.00%

